In [ ]:
!pip3 install -r ./drive/MyDrive/NLU\ Project/CODE/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import numpy as np

import transformers
import sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
from datasets import load_dataset
import ast
import time
import json

from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
load_dotenv()
api_key = os.getenv('your-key')
client = OpenAI(api_key='your-key')

In [ ]:
base_dir = "./drive/MyDrive/NLU Project/"

data = pd.read_csv(base_dir + 'DATA/nlu_data.csv')
data

,Unnamed: 0,Question,Context,Answer,Choice_1,Choice_2,Choice_3,Choice_4,Shuffled Context
0,0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,Saint Therese of Lisieux,Saint Bernadette Soubirous,Saint Catherine of Siena,Saint Joan of Arc,"In 2010, a genetic study was conducted on the ..."
1,1,When did the Scholastic Magazine of Notre dame...,"As at most other universities, Notre Dame's st...",September 1876,September 1876,January 1872,March 1880,August 1865,"On November 2, 2014, the Cubs announced that J..."
2,2,Where is the headquarters of the Congregation ...,The university is the major seat of the Congre...,Rome,Athens,Rome,Jerusalem,Mecca,"In higher education, Politecnico refers to a t..."
3,3,How many BS level degrees are offered in the C...,The College of Engineering was established in ...,eight,five,four,eight,two,While Southeast Asia is rich in flora and faun...
4,4,What entity provides help with the management ...,All of Notre Dame's undergraduate students are...,Learning Resource Center,Learning Support Facility,Student Success Hub,Career Development Center,Learning Resource Center,Baird Television Ltd. made Britain's first tel...
...,...,...,...,...,...,...,...,...,...
9995,9995,What was the film that Madonna starred in 2000?,"In 2000, Madonna starred in the film The Next ...",The Next Best Thing,The Next Best Thing,The Last Great Thing,The Perfect Opportunity,The Ultimate Solution,Seattle is considered the home of grunge music...
9996,9996,Who is Madonna's second husband?,"She met director Guy Ritchie, who would become...",Guy Ritchie,Actor Johnson,Director Smith,Guy Ritchie,Producer Brown,The economic reforms of President Carlos Salin...
9997,9997,Which film did Madonna starred in that was dir...,"Madonna starred in the film Swept Away, direct...",Swept Away,Swept Away,Wyndhams Theatre,Die Another Day,Up For Grabs,"Bell's patent 174,465, was issued to Bell on M..."
9998,9998,What is the name of the exhibition installatio...,"Following Die Another Day, Madonna collaborate...",X-STaTIC Pro=CeSS,X-STaTIC Pro=CeSS,X-STiC Pro=CESS,X-CPXTIC Pro=CeSS,X-STaTIX Pro=CeSS,"The most pelagic species, mainly in the 'tuben..."


In [ ]:
# Actually running Experiment 1
questions, real_contexts, shuffled_contexts, correct_answers, choices, correct_context_question_choice_combos, incorrect_context_question_choice_combos = [], [], [], [], [], [], []


total_rows = 1000

for ind in range(total_rows):
  row = data.iloc[ind]
  questions.append(row['Question'])
  real_contexts.append(row['Context'])
  shuffled_contexts.append(row['Shuffled Context'])
  correct_answers.append(row['Answer'])
  choices.append([row['Choice_1'], row['Choice_2'], row['Choice_3'], row['Choice_4']])

  options = {
      'A' : row['Choice_1'],
      'B' : row['Choice_2'],
      'C' : row['Choice_3'],
      'D' : row['Choice_4']
      }
  correct_context_question_choice_combos.append({
      'Context' : row['Context'],
      'Question' : row['Question'],
      'Options' : options})
  incorrect_context_question_choice_combos.append({
      'Context' : row['Shuffled Context'],
      'Question' : row['Question'],
      'Options' : options})

print(len(questions))

1000


In [ ]:
ind = 10
print(f'Question:\n{questions[ind]} \n\n\nReal Context:\n{real_contexts[ind]} \n\n\nShuffled Context:\n {shuffled_contexts[ind]} \n\n\nCorrect Answer:\n{correct_answers[ind]} \n\n\nChoices:\n{choices[ind]} \n\n\nCorrect_context_question_choice_combo:\n{correct_context_question_choice_combos[ind]} \n\n\nIncorrect_context_question_choice_combo:\n{incorrect_context_question_choice_combos[ind]}')
# print()

Question:
What person was the Director of the Science Museum at Notre Dame in the late 19th century? 


Real Context:
Father Joseph Carrier, C.S.C. was Director of the Science Museum and the Library and Professor of Chemistry and Physics until 1874. Carrier taught that scientific research and its promise for progress were not antagonistic to the ideals of intellectual and moral culture endorsed by the Church. One of Carrier's students was Father John Augustine Zahm (1851–1921) who was made Professor and Co-Director of the Science Department at age 23 and by 1900 was a nationally prominent scientist and naturalist. Zahm was active in the Catholic Summer School movement, which introduced Catholic laity to contemporary intellectual issues. His book Evolution and Dogma (1896) defended certain aspects of evolutionary theory as true, and argued, moreover, that even the great Church teachers Thomas Aquinas and Augustine taught something like it. The intervention of Irish American Catholics in

In [ ]:
base_messages = [{"role": "system", "content": """You have a question with four answer choices and a context which may or may not be associated with it. Your task is to return the correct answer option and answer. Reference delimited with ### to outline real examples of user input of contexts, questions, answers, and expected outputs.
######
RESPONSE STRUCTURE:
You must output your response as a JSON object following this structure:
{"answer_letter": {{correct answer letter (A, B, C, or D)}}, "answer_name": {{correct answer text}}, "explanation": {{explanation for why this is the correct answer}}}
######
"""}]

def call_api(model_input_args, model_name = "gpt-3.5-turbo"):
    '''
    Args:
    model_input_args : {
      "Context" : Str - context string
      "Question" : Str - Question String
      "Options" : Dict {'A' : Option_1,
                        'B' : Option_2,
                        'C' : Option_3,
                        'D' : Option_4
      }

    Returns:
      {'model_response' : model_response_string}

    '''
    global client, base_messages
    choices = model_input_args['Options']

    messages = base_messages + [{"role": "user", "content": f"""context: {model_input_args['Context']}\nquestion: {model_input_args['Question']}\nanswers: A. {choices['A']}\tB. {choices['B']}\tC. {choices['C']}\tD{choices['D']}"""}]

    attempts = 0
    while attempts < 3:
        try:
            # print(messages[1])
            response = client.chat.completions.create(model=model_name, messages=messages)
            # print('\n\nRESPONSE:\n', response)
            content = response.choices[0].message.content
            # print('\n\nCONTENT:\n', content)
            # print('\n\n\n\n\n\n\n\n\n\n\n\n')
            # content_dict = ast.literal_eval(content)
            # return content_dict
            return {content}
        except SyntaxError as e:
            if e == 'invalid syntax (<unknown>, line 1)':
                print(f"Error for model input \n{model_input_args}\n\n \'{e}\'")
                return {content}
            print(f"Error for model input \n{model_input_args}\n\n {e}")
            attempts += 1
            if attempts < 3:
                print("Retrying in 1 minute...")
                time.sleep(60)
            else:
                print("Failed after 3 attempts.")
                return {content}

In [ ]:
call_api(correct_context_question_choice_combos[0])

{'{"answer_letter": "B", "answer_name": "Saint Bernadette Soubirous", "explanation": "The Virgin Mary allegedly appeared to Saint Bernadette Soubirous in 1858 in Lourdes, France as referenced in the context provided."}'}

In [ ]:
correct_context_question_choice_combos[0]['Options']

{'A': 'Saint Therese of Lisieux',
 'B': 'Saint Bernadette Soubirous',
 'C': 'Saint Catherine of Siena',
 'D': 'Saint Joan of Arc'}

In [ ]:
correct_answers[0]

'Saint Bernadette Soubirous'

In [ ]:
def run_method_relevant_context(context_question_choice_combos, method = 'relevant'):
    """

    Args:
      context_question_choice_combos: List - [{
                                            "Context" : Str - context string
                                            "Question" : Str - Question String
                                            "Options" : Dict {'A' : Option_1,
                                                              'B' : Option_2,
                                                              'C' : Option_3,
                                                              'D' : Option_4
                                                              }
                                            ]
      method : Method to be run - relevant/ irrelevant/ none

    Returns:

    """
    responses = []
    if method == 'no_context':
        for i in context_question_choice_combos:
          i['Context'] = ''
    for i in tqdm(context_question_choice_combos):
      responses.append(call_api(i))


    return responses

In [ ]:
model_outputs_relevant = run_method_relevant_context(correct_context_question_choice_combos[:20], method = 'relevant')
model_outputs_relevant

100%|██████████| 20/20 [00:33<00:00,  1.69s/it]


[{'{"answer_letter": "B", "answer_name": "Saint Bernadette Soubirous", "explanation": "The Virgin Mary allegedly appeared to Saint Bernadette Soubirous in Lourdes, France in 1858. This is a well-known event in Catholic history and is associated with the Sanctuary of Our Lady of Lourdes."}'},
 {'{"answer_letter": "A", "answer_name": "September 1876", "explanation": "The Scholastic magazine of Notre Dame began publishing as a one-page journal in September 1876, making it the oldest continuous collegiate publication in the United States."}'},
 {'{"answer_letter": "B", "answer_name": "Rome", "explanation": "The headquarters of the Congregation of the Holy Cross is located in Rome, as mentioned in the context provided."}'},
 {'{"answer_letter": "C", "answer_name": "eight", "explanation": "The College of Engineering at Notre Dame offers eight B.S. degrees in aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer scienc

In [ ]:
correct_context_question_choice_combos[:20]

[{'Context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'Question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
  'Options': {'A': 'Saint Therese of Lisieux',
   'B': 'Saint Bernadette Soubirous',
   'C': 'Saint Catherine of Siena',
   'D': 'Saint Joan of Arc'}},
 {'Context': "As at most other universities, N

In [ ]:
list(model_outputs_relevant[0])[0]

'{"answer_letter": "B", "answer_name": "Saint Bernadette Soubirous", "explanation": "The Virgin Mary allegedly appeared to Saint Bernadette Soubirous in Lourdes, France in 1858. This is a well-known event in Catholic history and is associated with the Sanctuary of Our Lady of Lourdes."}'

In [ ]:
json_resp = json.loads(list(model_outputs_relevant[0])[0])
print(type(json_resp))

<class 'dict'>


In [ ]:
json_resp

{'answer_letter': 'B',
 'answer_name': 'Saint Bernadette Soubirous',
 'explanation': 'The Virgin Mary allegedly appeared to Saint Bernadette Soubirous in Lourdes, France in 1858. This is a well-known event in Catholic history and is associated with the Sanctuary of Our Lady of Lourdes.'}

In [ ]:
def check_match(model_output, options, correct_answer):
  """

  Args:
    model_output: Str - Model prediction according to the method being used
    options: Dict {'A' : Option_1,
                        'B' : Option_2,
                        'C' : Option_3,
                        'D' : Option_4
      }
    correct_answer: Str - Target prediction (Real Answer)

  Returns: True/False to evaluate model correctness

  """
  options = [options['A'], options['B'], options['C'], options['D']]
  correct_answer_index = options.index(correct_answer)

  try:
    model_output_json = json.loads(model_output)
    model_pred_option = model_output_json['answer_letter'].lower()

    if model_pred_option == 'abcd'[correct_answer_index]:
      return True
    else:
      return False
  except:
    matches = [options[i].lower() in model_output.lower() for i in range(4)]

    # matches has 4 elements, eg. for correct model response for correct answer being A: [True, False, False, False]
    # Here, we will flip only the value at the correct index i.e. it should go from True to False
    # If any other option matches with the model output or if the correct one doesn't match, then we return False,
    # If everything is exactly how it should be (model response is correct), then we return True
    matches[correct_answer_index] = not matches[correct_answer_index]

    # print(f'\n\n\nModel Output: \n{model_output}\n\nOptions: \n{options}\n\nCorrect Answer: {correct_answer}\n\nMatches List: {matches}')

    for i in matches:
      if i:
        return False
    return True





def calculate_accuracy(model_outputs, context_question_choice_combos, correct_answers):
  """

  Args:
    model_outputs: List <Str>
    context_question_choice_combos: List <Dict>

  Returns: Accuracy (# Correct / # Total)
  """

  count = 0
  n = len(model_outputs)
  for ind in range(n):
    model_output = list(model_outputs[ind])[0]
    # print(model_output, type(model_output))
    options = context_question_choice_combos[ind]['Options']
    correct_answer = correct_answers[ind]

    if check_match(model_output=model_output, options=options, correct_answer=correct_answer):
      count += 1

  return count/n

In [ ]:
calculate_accuracy(model_outputs_relevant, correct_context_question_choice_combos, correct_answers)

In [ ]:
model_outputs_irrelevant = run_method_relevant_context(incorrect_context_question_choice_combos[:20])
model_outputs_irrelevant

100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


[{'{"answer_letter": "B", "answer_name": "Saint Bernadette Soubirous", "explanation": "In 1858, it is alleged that the Virgin Mary appeared to Saint Bernadette Soubirous in Lourdes, France. Saint Bernadette Soubirous is known for her visions of the Virgin Mary in a grotto near Lourdes, which later became a major site for pilgrimage and healing."}'},
 {'{"answer_letter": "C", "answer_name": "March 1880", "explanation": "The Scholastic Magazine of Notre Dame began publishing in March 1880. This date is unrelated to the context provided."}'},
 {'{"answer_letter": "B", "answer_name": "Rome", "explanation": "The headquarters of the Congregation of the Holy Cross is located in Rome, Italy."}'},
 {'{"answer_letter": "D", "answer_name": "two", "explanation": "The question provided is not related to the context about deforestation and haze in Southeast Asia. The correct answer is \'two\' as per the question given."}'},
 {'{"answer_letter": "B", "answer_name": "Student Success Hub", "explanation

In [ ]:
incorrect_context_question_choice_combos[:20]

[{'Context': "In 2010, a genetic study was conducted on the mummified remains of baboons that were brought back as gifts from Punt by the ancient Egyptians. Led by a research team from the Egyptian Museum and the University of California, the scientists used oxygen isotope analysis to examine hairs from two baboon mummies that had been preserved in the British Museum. One of the baboons had distorted isotopic data, so the other's oxygen isotope values were compared to those of present-day baboon specimens from regions of interest. The researchers found that the mummies most closely matched modern baboon specimens in Eritrea and Ethiopia, which they suggested implied that Punt was likely a narrow region that included eastern Ethiopia and all of Eritrea.",
  'Question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
  'Options': {'A': 'Saint Therese of Lisieux',
   'B': 'Saint Bernadette Soubirous',
   'C': 'Saint Catherine of Siena',
   'D': 'Saint Joan of Ar

In [ ]:
# Irrelevant Context Method Accuracy
calculate_accuracy(model_outputs_irrelevant, incorrect_context_question_choice_combos, correct_answers)

In [ ]:
# No context accuracy.
model_outputs_no_context = run_method_relevant_context(correct_context_question_choice_combos[:20], method = 'no_context')
model_outputs_no_context

100%|██████████| 20/20 [00:27<00:00,  1.36s/it]


[{'{"answer_letter": "B", "answer_name": "Saint Bernadette Soubirous", "explanation": "In 1858, the Virgin Mary allegedly appeared to Saint Bernadette Soubirous in Lourdes, France. Saint Bernadette\'s vision led to the creation of the Sanctuary of Our Lady of Lourdes, which is now one of the major Catholic pilgrimage sites in the world."}'},
 {'{"answer_letter": "B", "answer_name": "January 1872", "explanation": "The Scholastic Magazine of Notre Dame began publishing in January 1872, making it the correct answer."}'},
 {'{"answer_letter": "B", "answer_name": "Rome", "explanation": "The headquarters of the Congregation of the Holy Cross is located in Rome, Italy."}'},
 {'{"answer_letter": "A", "answer_name": "five", "explanation": "The College of Engineering at Notre Dame offers five BS level degrees."}'},
 {'{"answer_letter": "B", "answer_name": "Student Success Hub", "explanation": "The Student Success Hub at Notre Dame provides help with the management of time for new students, offer

In [ ]:
correct_context_question_choice_combos[:20]

[{'Context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
  'Question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
  'Options': {'A': 'Saint Therese of Lisieux',
   'B': 'Saint Bernadette Soubirous',
   'C': 'Saint Catherine of Siena',
   'D': 'Saint Joan of Arc'}},
 {'Context': "As at most other universities, N

In [ ]:
# No Context Method Accuracy
calculate_accuracy(model_outputs_no_context, correct_context_question_choice_combos, correct_answers)